In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from scipy.interpolate import make_interp_spline

# Check for GPU availability and configure TensorFlow to use the GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Using GPU: ", gpus[0])
    except RuntimeError as e:
        print(e)

# Constants
mu_0 = 4 * np.pi * 1e-7  # Permeability of free space (H/m)
periods = np.logspace(-3, 3, 25)  # 25 periods ranging from 0.001 s to 10 s
resistivity_range = [1, 10000]  # Min and max resistivities in Ohm.m
rho_ref = 100  # Reference resistivity in Ohm.m

# Calculate min and max skin depths
min_period = periods.min()
max_period = periods.max()
min_skin_depth = 500 * np.sqrt(rho_ref * min_period) / 4
max_skin_depth = 500 * np.sqrt(rho_ref * max_period)

# Calculate layer thicknesses
layer_thicknesses = [min_skin_depth]
while layer_thicknesses[-1] * 1.2 < max_skin_depth:
    layer_thicknesses.append(layer_thicknesses[-1] * 1.2)
layer_thicknesses = np.array(layer_thicknesses)
num_layers = len(layer_thicknesses)

def generate_smooth_resistivity_profile(num_layers, resistivity_range):
    # Generate a few points to define the smooth profile
    depth_points = np.linspace(0, num_layers, num=10)
    resistivity_points = np.random.uniform(resistivity_range[0], resistivity_range[1], len(depth_points))

    # Create a cubic spline interpolation for smooth resistivity
    spline = make_interp_spline(depth_points, resistivity_points, k=3)
    smooth_resistivities = spline(np.linspace(0, num_layers, num_layers))

    return smooth_resistivities

def compute_apparent_resistivity_and_phase(thicknesses, conductivities, periods):
    apparent_resistivity = []
    phase = []

    for T in periods:
        omega = 2 * np.pi / T  # Angular frequency (rad/s)
        cns = np.zeros(len(conductivities), dtype=complex)
        cns[-1] = 1 / np.sqrt(mu_0 * omega * conductivities[-1] * 1j)

        for j in reversed(range(len(thicknesses))):
            K = np.sqrt(conductivities[j] * mu_0 * omega * 1j)
            layer_thickness = thicknesses[j] if j < len(thicknesses) - 1 else np.inf  # last layer thickness is unknown
            if j + 1 < len(cns):
                cns[j] = (1 / K) * ((K * cns[j + 1] + np.tanh(K * layer_thickness)) /
                                    (1 + K * cns[j + 1] * np.tanh(K * layer_thickness)))

        Z = cns[0]
        rho_apparent = np.abs(Z)**2 * (mu_0 * omega)
        phi = np.angle(Z, deg=True) + 90  # Calculate phase correctly using np.angle

        apparent_resistivity.append(rho_apparent)
        phase.append(phi)

    return np.array(apparent_resistivity), np.array(phase)

# Generate synthetic dataset for 100,000 stations
num_stations = 100000
X_model = []
y_rho = []
y_phi = []

for _ in range(num_stations):
    # Generate synthetic resistivity profile
    resistivities = generate_smooth_resistivity_profile(num_layers, resistivity_range)
    conductivities = 1 / resistivities

    # Compute apparent resistivity and phase using the analytical solution
    apparent_resistivity, phase = compute_apparent_resistivity_and_phase(layer_thicknesses, conductivities, periods)

    # Store the data
    X_model.append(resistivities)
    y_rho.append(apparent_resistivity)
    y_phi.append(phase)

# Convert lists to numpy arrays
X_model = np.array(X_model)
y_rho = np.array(y_rho)
y_phi = np.array(y_phi)

# Normalize data
scaler_X_model = MinMaxScaler()
scaler_y_rho = MinMaxScaler()
scaler_y_phi = MinMaxScaler()

X_model_scaled = scaler_X_model.fit_transform(X_model)
y_rho_scaled = scaler_y_rho.fit_transform(y_rho)
y_phi_scaled = scaler_y_phi.fit_transform(y_phi)

# Combine y_rho and y_phi for the input data
X_combined = np.hstack((y_rho_scaled, y_phi_scaled))

# Neural network model for MT inversion
def build_nn(input_shape):
    inputs = Input(shape=input_shape)

    x = Dense(128, activation='relu')(inputs)
    x = Dense(256, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(num_layers)(x)

    model = Model(inputs, outputs)
    return model

model = build_nn((X_combined.shape[1],))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Split data into training and validation sets (80/20 ratio)
X_train, X_val, y_train, y_val = train_test_split(X_combined, X_model_scaled, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Predict on the validation data
y_pred = model.predict(X_val)

# Inverse transform the predictions and actual values
y_pred_resistivity = scaler_X_model.inverse_transform(y_pred)
y_val_resistivity = scaler_X_model.inverse_transform(y_val)

# Compare the predicted resistivity profiles with the true profiles
num_stations_to_plot = 10  # Number of stations to plot

fig, axs = plt.subplots(num_stations_to_plot, 1, figsize=(10, num_stations_to_plot*5), dpi=300)
for i in range(num_stations_to_plot):
    depths = np.cumsum(layer_thicknesses)

    # Plot true and predicted resistivity profiles
    ax = axs[i]
    ax.step(np.insert(depths, 0, 0), np.insert(y_val_resistivity[i], 0, y_val_resistivity[i][0]), where='post', label='True Resistivity')
    ax.step(np.insert(depths, 0, 0), np.insert(y_pred_resistivity[i], 0, y_pred_resistivity[i][0]), where='post', label='Predicted Resistivity')
    ax.set_xlabel('Depth (m)')
    ax.set_ylabel('Resistivity (Ohm-m)')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_title(f'Resistivity vs Depth: Station {i+1}')
    ax.legend()

plt.tight_layout()
plt.show()

# Plot the training and validation loss over epochs
plt.figure(figsize=(10, 6), dpi=300)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.legend()
plt.tight_layout()
plt.show()
